In [3]:
import math
import random
import queue

In [4]:
def Exp(mean):
    lamda = 1 / mean
    return -1 * math.log(1 - random.random()) / lamda

def classicProbability(probability):
    val = random.random() * 100 
    return True if val < probability else False

def ArrivalTime():
    return Exp(1/20) * 60 

def RServiceTime():
    return Exp(4)

def PServiceTime():
    return Exp(5) 

def FromPToR():
    return classicProbability(10)
    
def FromRToP():
    return classicProbability(20)

def POrR():
    return classicProbability(40)

In [100]:
count = 0

for i in range(1000):
    count += PServiceTime()
print(count / 1000)

4.965739431465378


In [233]:
def Sim(ArrivalTime, RServiceTime, PServiceTime, FromPToR, FromRToP, POrR, maxTime):
    time = 0
    aT = ArrivalTime()
    dTP = maxTime + 1
    dTR = maxTime + 1
    clientCount = 0
    queueCountAvg = 0
    queueCount = 0
    
    qR = [] #volver una cola
    qP = [] #volver una cola

    isFreeP = True
    isFreeR = True
    clientOnP = -1
    clientOnR = -1
    
    eventList = []
    clientInfo = {}
    
    while time <= maxTime:
        _min = min(aT, dTR, dTP)    
        
        if _min == aT:
            time = aT
            aT = ArrivalTime() + time
            
            if POrR():
                clientCount += 1
                eventList.append(("LLegada a la cola del pollo frito", time, clientCount))
                clientInfo[clientCount] = {"arrival": time, "departure" : -1, "both" : False }
                qP.append(clientCount)
                queueCountAvg += len(qP)
                queueCount += 1
                
                if isFreeP:
                    dTP = time + PServiceTime()
                    clientOnP = qP.pop(0)
                    eventList.append(("LLegada a la ventana del pollo frito", time, clientOnP))
                    isFreeP = False
                
            else:
                clientCount += 1
                eventList.append(("LLegada a la cola de los rollitos", time, clientCount))
                clientInfo[clientCount] = {"arrival": time, "departure" : -1, "both" : False}
                qR.append(clientCount)
                queueCountAvg += len(qR)
                queueCount += 1
                
                if isFreeR:
                    dTR = time + RServiceTime()
                    clientOnR = qR.pop(0)
                    eventList.append(("LLegada a la ventana de los rollitos", time, clientOnR))
                    isFreeR = False
                
        elif _min == dTP:
            time = dTP
            eventList.append(("Salida de la ventana del pollo frito", time, clientOnP))
            
            if not clientInfo[clientOnP]["both"] and FromPToR():
                eventList.append(("Cambio a la cola de rollitos", time, clientOnP))
                clientInfo[clientOnP]["both"] = True
                qR.append(clientOnP)
            else:
                clientInfo[clientOnP]["departure"] = time
                queueCountAvg += len(qP)
                queueCount += 1
                
            if len(qP) > 0:
                dTP = time + PServiceTime()
                clientOnP = qP.pop(0)
                eventList.append(("LLegada a la ventana del pollo frito", time, clientOnP))
                 
            else:
                dTP = maxTime + 1
                isFreeP = True 
            
        elif _min == dTR:
            time = dTR
            eventList.append(("Salida de la ventana de los rollitos", time, clientOnR))
            
            if not clientInfo[clientOnR]["both"] and FromRToP():
                eventList.append(("Cambio a la cola del pollo frito", time, clientOnR))
                clientInfo[clientOnR]["both"] = True
                qP.append(clientOnR)
                
            else:
                clientInfo[clientOnR]["departure"] = time
                queueCountAvg += len(qR)
                queueCount += 1
            
            if len(qR) > 0:
                dTR = time + RServiceTime()
                clientOnR = qR.pop(0)
                eventList.append(("LLegada a la ventana de los rollitos", time, clientOnR))
                
            else:
                dTR = maxTime + 1
                isFreeR = True 
    
    waitTimeAvg = 0
    waitTimeBothAvg = 0
    bothCount = 0
    count = 0
    
    for client in clientInfo.values():
        if not client["departure"] == -1:
            waitTime = client["departure"] - client["arrival"]
            waitTimeAvg += waitTime
            count += 1
            
            if client["both"]:
                
                waitTimeBothAvg += waitTime
                bothCount += 1
            
    
    waitTimeAvg /= count
    waitTimeBothAvg /= bothCount
    queueCountAvg /= queueCount
    
    return clientCount, waitTimeAvg, waitTimeBothAvg, queueCountAvg

In [234]:
print(Sim(ArrivalTime, RServiceTime, PServiceTime, FromPToR, FromRToP, POrR, 1440))

iters = 1000
totalWeek = 0
waitTimeAvgAvg = 0
waitTimeBothAvgAvg = 0
queueCountAvgAvg = 0

for i in range(iters): 
    count, waitTimeAvg, waitTimeBothAvg, queueCountAvg = Sim(ArrivalTime, RServiceTime, PServiceTime, FromPToR, FromRToP, POrR, 10080)
    totalWeek += count
    waitTimeAvgAvg += waitTimeAvg
    waitTimeBothAvgAvg += waitTimeBothAvg
    queueCountAvgAvg += queueCountAvg
    
print(totalWeek / iters, 
    waitTimeAvgAvg / iters,
    waitTimeBothAvgAvg / iters,
    queueCountAvgAvg / iters)

totalDay = 0

waitTimeAvgAvg = 0
waitTimeBothAvgAvg = 0
queueCountAvgAvg = 0
for i in range(iters * 7): 
    count, waitTimeAvg, waitTimeBothAvg, queueCountAvg = Sim(ArrivalTime, RServiceTime, PServiceTime, FromPToR, FromRToP, POrR, 1440)
    totalDay += count
    waitTimeAvgAvg += waitTimeAvg
    waitTimeBothAvgAvg += waitTimeBothAvg
    queueCountAvgAvg += queueCountAvg
    
print(totalDay / (iters),   
    waitTimeAvgAvg / (7*iters),
    waitTimeBothAvgAvg / (7*iters),
    queueCountAvgAvg / (7*iters))


(479, 30.235265497605013, 54.803041384605805, 5.366807610993657)
3359.9 37.217823960619434 65.97684777075816 6.2308872212134805
3361.87 31.69987377775403 56.278812982791244 5.335845825506594
